# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-51bde93817-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Literary Movements and physicists explorative search") 

Consider the following exploratory information need:

> investigate the literary movements and the poets. In particular, compare some physicists in terms of discovers, Nobel Prizes and education.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q43514`| theory of relativity | node |





Also consider

```
wd:Q43514 ?p ?obj .
```

is the BGP to retrieve all **properties of the theory of relativity**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:

1. Who invented the theory of relativity? (return the IRI and the label of the inventor).

2. Identify the BGP for physicist

3. Identify the BGP for Nobel Prize

4. Consider physicists born in Europe. For each european country return the number of physicists who won the Nobel Prize in Physics (the result set must be a list of triples country IRI, label and #physicists).

5. Return the countries which won at least once all the type of Nobel Prizes with the people born there. (the result set must be a list of couples country IRI and label). 

6. Amongst the physicists who won the Nobel Prize in Physics, find who had discovered or invented at least 3 things (the result set must be a list of triples physicist IRI, label and #discoveries ).

7. Amongst the physicists who won the Nobel Prize in Physics, find who taught in the university of his/her city of birth (the result set must be a list of elements of the form physicist IRI, label, university IRI and label).

## Task 1
Who invented the theory of relativity? (return the IRI and the label of the inventor).

In [2]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q43514 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('name', 'has quality')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('

In [2]:
# retrieve more predicates about the theory of relativity
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q43514 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('name', 'has quality')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('

In [5]:
# use the predicate P61 discoverer or inventor
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q43514 wdt:P61 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q937'), ('name', 'Albert Einstein')]
1


Final query for this task

In [6]:
# write your final query
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q43514 wdt:P61 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q937'), ('name', 'Albert Einstein')]
1


## Task 2
Identify the BGP for physicist

In [9]:
# write your queries
# I'll start by looking at wdt:P106 occupation
queryString = """
SELECT DISTINCT ?s ?sname ?obj ?objname
WHERE {
   # bind something
   ?s wdt:P106 ?obj .
   # get the label
   ?s sc:name ?sname .
   ?obj sc:name ?objname.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q8200247'), ('sname', 'Annette Aiello'), ('obj', 'http://www.wikidata.org/entity/Q12368980'), ('objname', 'nature photographer')]
[('s', 'http://www.wikidata.org/entity/Q8186437'), ('sname', 'Buzz Feitshans IV'), ('obj', 'http://www.wikidata.org/entity/Q27890911'), ('objname', 'second unit director')]
[('s', 'http://www.wikidata.org/entity/Q13639581'), ('sname', 'John Ceiriog Hughes'), ('obj', 'http://www.wikidata.org/entity/Q2962020'), ('objname', 'station master')]
[('s', 'http://www.wikidata.org/entity/Q15485380'), ('sname', 'Mary Louise Curtis Bok Zimbalist'), ('obj', 'http://www.wikidata.org/entity/Q19435686'), ('objname', 'founder')]
[('s', 'http://www.wikidata.org/entity/Q15457166'), ('sname', 'Karl von Grothaus'), ('obj', 'http://www.wikidata.org/entity/Q221488'), ('objname', 'pedestrian')]
[('s', 'http://www.wikidata.org/entity/Q13639581'), ('sname', 'John Ceiriog Hughes'), ('obj', 'http://www.wikidata.org/entity/Q738142'), ('objn

In [11]:
# try to retrieve all the physicist
queryString = """
SELECT DISTINCT ?s ?sname ?obj ?objname
WHERE {
   # bind something
   ?s wdt:P106 ?obj .
   # get the label
   ?s sc:name ?sname .
   ?obj sc:name ?objname.
   FILTER REGEX (?objname, "physicist")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q16013709'), ('sname', 'John Elliott Nafe'), ('obj', 'http://www.wikidata.org/entity/Q12094958'), ('objname', 'geophysicist')]
[('s', 'http://www.wikidata.org/entity/Q16019342'), ('sname', 'John Call Cook'), ('obj', 'http://www.wikidata.org/entity/Q12094958'), ('objname', 'geophysicist')]
[('s', 'http://www.wikidata.org/entity/Q1677140'), ('sname', 'Jack Oliver'), ('obj', 'http://www.wikidata.org/entity/Q12094958'), ('objname', 'geophysicist')]
[('s', 'http://www.wikidata.org/entity/Q7183518'), ('sname', 'Philip England'), ('obj', 'http://www.wikidata.org/entity/Q12094958'), ('objname', 'geophysicist')]
[('s', 'http://www.wikidata.org/entity/Q1683869'), ('sname', 'Marcia McNutt'), ('obj', 'http://www.wikidata.org/entity/Q12094958'), ('objname', 'geophysicist')]
[('s', 'http://www.wikidata.org/entity/Q29107821'), ('sname', 'Huiming Bao'), ('obj', 'http://www.wikidata.org/entity/Q12094958'), ('objname', 'geophysicist')]
[('s', 'http://www.wi

In [14]:
# retrieve all the jobs as physicist 
queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {
   # bind something
   ?s wdt:P106 ?obj .
   # get the label
   ?obj sc:name ?objname.
   FILTER REGEX (?objname, "physicist")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q12094958'), ('objname', 'geophysicist')]
[('obj', 'http://www.wikidata.org/entity/Q19350898'), ('objname', 'theoretical physicist')]
[('obj', 'http://www.wikidata.org/entity/Q752129'), ('objname', 'astrophysicist')]
[('obj', 'http://www.wikidata.org/entity/Q29909337'), ('objname', 'experimental physicist')]
[('obj', 'http://www.wikidata.org/entity/Q14906342'), ('objname', 'biophysicist')]
[('obj', 'http://www.wikidata.org/entity/Q16742096'), ('objname', 'nuclear physicist')]
[('obj', 'http://www.wikidata.org/entity/Q169470'), ('objname', 'physicist')]
[('obj', 'http://www.wikidata.org/entity/Q1916621'), ('objname', 'medical physicist')]
[('obj', 'http://www.wikidata.org/entity/Q16745601'), ('objname', 'quantum physicist')]
[('obj', 'http://www.wikidata.org/entity/Q63017728'), ('objname', 'particle physicist')]
[('obj', 'http://www.wikidata.org/entity/Q91006792'), ('objname', 'atmospheric physicist')]
[('obj', 'http://www.wikidata.org/en

Final query for this task

In [15]:
# write your final query
queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {
   # bind something
   ?s wdt:P106 ?obj .
   # get the label
   ?obj sc:name ?objname.
   FILTER REGEX (?objname, "physicist")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q12094958'), ('objname', 'geophysicist')]
[('obj', 'http://www.wikidata.org/entity/Q19350898'), ('objname', 'theoretical physicist')]
[('obj', 'http://www.wikidata.org/entity/Q752129'), ('objname', 'astrophysicist')]
[('obj', 'http://www.wikidata.org/entity/Q29909337'), ('objname', 'experimental physicist')]
[('obj', 'http://www.wikidata.org/entity/Q14906342'), ('objname', 'biophysicist')]
[('obj', 'http://www.wikidata.org/entity/Q16742096'), ('objname', 'nuclear physicist')]
[('obj', 'http://www.wikidata.org/entity/Q169470'), ('objname', 'physicist')]
[('obj', 'http://www.wikidata.org/entity/Q1916621'), ('objname', 'medical physicist')]
[('obj', 'http://www.wikidata.org/entity/Q16745601'), ('objname', 'quantum physicist')]
[('obj', 'http://www.wikidata.org/entity/Q63017728'), ('objname', 'particle physicist')]
[('obj', 'http://www.wikidata.org/entity/Q91006792'), ('objname', 'atmospheric physicist')]
[('obj', 'http://www.wikidata.org/en

## Task 3
Identify the BGP for Nobel Prize

In [16]:
# write your queries
# I'll start by looking at Q937 ALbert Einstein predicates and objs
queryString = """
SELECT DISTINCT ?p ?pname ?obj ?objname
WHERE {
   # bind something
   wd:Q937 ?p ?obj .
   # get the label
   ?p sc:name ?pname.
   ?obj sc:name ?objname.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P551'), ('pname', 'residence'), ('obj', 'http://www.wikidata.org/entity/Q1309294'), ('objname', 'Einsteinhaus Caputh')]
[('p', 'http://www.wikidata.org/prop/direct/P800'), ('pname', 'notable work'), ('obj', 'http://www.wikidata.org/entity/Q11452'), ('objname', 'general relativity')]
[('p', 'http://www.wikidata.org/prop/direct/P800'), ('pname', 'notable work'), ('obj', 'http://www.wikidata.org/entity/Q944'), ('objname', 'quantum mechanics')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pname', 'educated at'), ('obj', 'http://www.wikidata.org/entity/Q435651'), ('objname', 'old Kantonsschule (Albert Einstein House)')]
[('p', 'http://www.wikidata.org/prop/direct/P5021'), ('pname', 'assessment'), ('obj', 'http://www.wikidata.org/entity/Q1051792'), ('objname', 'intelligence test')]
[('p', 'http://www.wikidata.org/prop/direct/P20'), ('pname', 'place of death'), ('obj', 'http://www.wikidata.org/entity/Q138518'), ('objname', 'Princeton

In [17]:
# I have found that Q38104 is the Nobel Prize in Physics, so I'll look at it subclassOf
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q38104 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3553')

In [18]:
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q38104 wdt:P279 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q7191'), ('name', 'Nobel Prize')]
1


Final query for this task

In [19]:
# write your final query
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q38104 wdt:P279 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q7191'), ('name', 'Nobel Prize')]
1


## Task 4
Consider physicists born in Europe. For each european country return the number of physicists who won the Nobel Prize in Physics (the result set must be a list of triples country IRI, label and #physicists).

In [20]:
# write your queries
# I'll use the same query I have used on the first notebook to retrieve the european countries.
queryString = """
SELECT DISTINCT ?europeanCountries ?countryName
WHERE {
   # bind something
   ?europeanCountries wdt:P31 wd:Q6256 ;
               wdt:P30 wd:Q46 .
   # get the label
   ?europeanCountries sc:name ?countryName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('europeanCountries', 'http://www.wikidata.org/entity/Q756617'), ('countryName', 'Danish Realm')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q907112'), ('countryName', 'Transnistria')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q184'), ('countryName', 'Belarus')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q20'), ('countryName', 'Norway')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q211'), ('countryName', 'Latvia')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q213'), ('countryName', 'Czech Republic')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q214'), ('countryName', 'Slovakia')]
[('europeanCountries', 'http://www.wikidata.org/entity/Q218'), ('countryName', 'Romania')]
[('europeanCountries', 'http://www.wikidata.org/entity/

In [21]:
# I'll try to retrieve the country of born of an Human wd:Q5
queryString = """
SELECT DISTINCT ?p ?pname ?obj ?objname
WHERE {
   # bind something
   wd:Q5 ?p ?obj .
   # get the label
   ?p sc:name ?pname.
   ?obj sc:name ?objname.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P1317'), ('objname', 'floruit')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P1477'), ('objname', 'birth name')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P1559'), ('objname', 'name in native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P509'), ('objname', 'cause of death')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P3373'), ('objname', 'sibling')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('pname', 'has quality'), ('obj', 'http://www.wikidata.org/entity/Q785260'), ('objname', 

In [22]:
queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {
   # bind something
   wd:Q5 wdt:P19 ?obj .
   # get the label
   ?obj sc:name ?objname.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [23]:
queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {
   # bind something
   wd:Q5 wd:P19 ?obj .
   # get the label
   ?obj sc:name ?objname.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [25]:
# this is a wrong query
queryString = """
SELECT DISTINCT ?europeanCountry ?countryName ?obj ?name
WHERE {
   # bind something
   ?europeanCountry wdt:P31 wd:Q6256 ;
               wdt:P30 wd:Q46 .
   wd:Q5 ?p ?obj .
   # get the label
   ?europeanCountry sc:name ?countryName .
   ?obj sc:name ?name .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('europeanCountry', 'http://www.wikidata.org/entity/Q756617'), ('countryName', 'Danish Realm'), ('obj', 'http://www.wikidata.org/entity/Q785260'), ('name', 'phase of human life')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q756617'), ('countryName', 'Danish Realm'), ('obj', 'http://www.wikidata.org/entity/Q95074'), ('name', 'fictional character')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q756617'), ('countryName', 'Danish Realm'), ('obj', 'http://www.wikidata.org/entity/Q3769299'), ('name', 'human behavior')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q756617'), ('countryName', 'Danish Realm'), ('obj', 'http://www.wikidata.org/entity/Q215627'), ('name', 'person')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q756617'), ('countryName', 'Danish Realm'), ('obj', 'http://www.wikidata.org/entity/Q1071027'), ('name', 'personal name')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q756617'), ('countryName', 'Danish Realm'), ('obj', 'http

In [26]:
queryString = """
SELECT DISTINCT ?europeanCountry ?countryName ?obj ?name
WHERE {
   # bind something
   ?europeanCountry wdt:P31 wd:Q6256 ;
               wdt:P30 wd:Q46 .
   wd:Q5 ?p ?europeanCountry .
   # get the label
   ?europeanCountry sc:name ?countryName .
   ?obj sc:name ?name .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [27]:
# I'll restart by retrieving all the winner of Q38104 Nobel Prize in Physics P166
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104 .
   # get the label
   ?europeanCountry sc:name ?countryName .
   ?s sc:name ?name .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q15900993'), ('name', 'Barry C. Barish')]
[('s', 'http://www.wikidata.org/entity/Q76766'), ('name', 'Gerd Binnig')]
[('s', 'http://www.wikidata.org/entity/Q192708'), ('name', 'Robert Coleman Richardson')]
[('s', 'http://www.wikidata.org/entity/Q57072'), ('name', 'Otto Stern')]
[('s', 'http://www.wikidata.org/entity/Q71024'), ('name', 'Klaus von Klitzing')]
[('s', 'http://www.wikidata.org/entity/Q65807'), ('name', 'Reinhard Genzel')]
[('s', 'http://www.wikidata.org/entity/Q71022'), ('name', 'Ernst Ruska')]
[('s', 'http://www.wikidata.org/entity/Q190495'), ('name', 'Henry Way Kendall')]
[('s', 'http://www.wikidata.org/entity/Q71023'), ('name', 'Alfred Kastler')]
[('s', 'http://www.wikidata.org/entity/Q191922'), ('name', 'Frederick Reines')]
[('s', 'http://www.wikidata.org/entity/Q57063'), ('name', 'Philipp Lenard')]
[('s', 'http://www.wikidata.org/entity/Q57067'), ('name', 'Max von Laue')]
[('s', 'http://www.wikidata.org/entity/Q57068'), ('n

In [28]:
# Now I'll retrieve the place of birth of each of them
queryString = """
SELECT DISTINCT ?s ?name ?place ?placeName
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104 ;
       wdt:P19 ?place .
   # get the label
   ?place sc:name ?placeName .
   ?s sc:name ?name .
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q15900993'), ('name', 'Barry C. Barish'), ('place', 'http://www.wikidata.org/entity/Q43199'), ('placeName', 'Omaha')]
[('s', 'http://www.wikidata.org/entity/Q76766'), ('name', 'Gerd Binnig'), ('place', 'http://www.wikidata.org/entity/Q1794'), ('placeName', 'Frankfurt am Main')]
[('s', 'http://www.wikidata.org/entity/Q192708'), ('name', 'Robert Coleman Richardson'), ('place', 'http://www.wikidata.org/entity/Q61'), ('placeName', 'Washington, D.C.')]
[('s', 'http://www.wikidata.org/entity/Q57072'), ('name', 'Otto Stern'), ('place', 'http://www.wikidata.org/entity/Q393171'), ('placeName', 'Żory')]
[('s', 'http://www.wikidata.org/entity/Q71024'), ('name', 'Klaus von Klitzing'), ('place', 'http://www.wikidata.org/entity/Q554658'), ('placeName', 'Gmina Środa Wielkopolska')]
[('s', 'http://www.wikidata.org/entity/Q65807'), ('name', 'Reinhard Genzel'), ('place', 'http://www.wikidata.org/entity/Q4165'), ('placeName', 'Bad Homburg vor der Höhe')]
[('

In [29]:
# Now I'll retrieve the country of the place of birth
queryString = """
SELECT DISTINCT ?place ?placeName ?p ?pName ?country ?countryName
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104 ;
       wdt:P19 ?place .
   ?place ?p ?country .
   # get the label
   ?place sc:name ?placeName .
   ?p sc:name ?pName .
   ?country sc:name ?countryName .
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('place', 'http://www.wikidata.org/entity/Q932261'), ('placeName', 'Sydney'), ('p', 'http://www.wikidata.org/prop/direct/P112'), ('pName', 'founded by'), ('country', 'http://www.wikidata.org/entity/Q3184971'), ('countryName', 'Joseph Frederick Wallet Des Barres')]
[('place', 'http://www.wikidata.org/entity/Q72'), ('placeName', 'Zürich'), ('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pName', "topic's main Wikimedia portal"), ('country', 'http://www.wikidata.org/entity/Q11384514'), ('countryName', 'Portal:Zürich')]
[('place', 'http://www.wikidata.org/entity/Q28180'), ('placeName', 'Jefferson City'), ('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('country', 'http://www.wikidata.org/entity/Q484493'), ('countryName', 'Cole County')]
[('place', 'http://www.wikidata.org/entity/Q43199'), ('placeName', 'Omaha'), ('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territor

In [30]:
queryString = """
SELECT DISTINCT ?place ?placeName ?p ?pName
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104 ;
       wdt:P19 ?place .
   ?place ?p ?obj .
   # get the label
   ?place sc:name ?placeName .
   ?p sc:name ?pName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('place', 'http://www.wikidata.org/entity/Q10727452'), ('placeName', 'Örebro Nikolai parish'), ('p', 'http://www.wikidata.org/prop/direct/P5324'), ('pName', 'Swedish National Archive Reference Code')]
[('place', 'http://www.wikidata.org/entity/Q10727452'), ('placeName', 'Örebro Nikolai parish'), ('p', 'http://www.wikidata.org/prop/direct/P7607'), ('pName', 'WikiTree category or space')]
[('place', 'http://www.wikidata.org/entity/Q10727452'), ('placeName', 'Örebro Nikolai parish'), ('p', 'http://www.wikidata.org/prop/direct/P778'), ('pName', 'Church of Sweden parish code')]
[('place', 'http://www.wikidata.org/entity/Q10727452'), ('placeName', 'Örebro Nikolai parish'), ('p', 'http://www.wikidata.org/prop/direct/P779'), ('pName', 'Church of Sweden Pastoratskod')]
[('place', 'http://www.wikidata.org/entity/Q187346'), ('placeName', 'Constantine'), ('p', 'http://www.wikidata.org/prop/direct/P1017'), ('pName', 'Vatican Library ID (former scheme)')]
[('place', 'http://www.wikidata.org

In [31]:
# I'll get all the predicates and objs of Genoa Q1449 
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q1449 ?p ?obj .
   # get the label
   ?p sc:name ?pName .
   ?obj sc:name ?objName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('pName', 'owner of'), ('obj', 'http://www.wikidata.org/entity/Q15845585'), ('objName', 'Villa Spinola di San Pietro')]
[('p', 'http://www.wikidata.org/prop/direct/P1376'), ('pName', 'capital of'), ('obj', 'http://www.wikidata.org/entity/Q174306'), ('objName', 'Republic of Genova')]
[('p', 'http://www.wikidata.org/prop/direct/P190'), ('pName', 'twinned administrative body'), ('obj', 'http://www.wikidata.org/entity/Q52337'), ('objName', 'Tursi')]
[('p', 'http://www.wikidata.org/prop/direct/P190'), ('pName', 'twinned administrative body'), ('obj', 'http://www.wikidata.org/entity/Q57948'), ('objName', 'Aue')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('obj', 'http://www.wikidata.org/entity/Q18288152'), ('objName', 'Metropolitan City of Genoa')]
[('p', 'http://www.wikidata.org/prop/direct/P1376'), ('pName', 'capital of'), ('obj', 'http://www.wikidata.org/ent

In [33]:
# Now I'll retrieve the country of the place of birth
queryString = """
SELECT DISTINCT ?place ?placeName ?europeanCountry ?countryName
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104 ;
       wdt:P19 ?place .
   ?europeanCountry wdt:P31 wd:Q6256 ;
               wdt:P30 wd:Q46 .
   ?place wdt:P17 ?europeanCountry .
   # get the label
   ?place sc:name ?placeName .
   ?europeanCountry sc:name ?countryName .
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('place', 'http://www.wikidata.org/entity/Q10727452'), ('placeName', 'Örebro Nikolai parish'), ('europeanCountry', 'http://www.wikidata.org/entity/Q34'), ('countryName', 'Sweden')]
[('place', 'http://www.wikidata.org/entity/Q1908426'), ('placeName', 'Bearsden'), ('europeanCountry', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom')]
[('place', 'http://www.wikidata.org/entity/Q1449'), ('placeName', 'Genoa'), ('europeanCountry', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy')]
[('place', 'http://www.wikidata.org/entity/Q22905'), ('placeName', 'Bradford'), ('europeanCountry', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom')]
[('place', 'http://www.wikidata.org/entity/Q72'), ('placeName', 'Zürich'), ('europeanCountry', 'http://www.wikidata.org/entity/Q39'), ('countryName', 'Switzerland')]
[('place', 'http://www.wikidata.org/entity/Q102217'), ('placeName', 'Vitebsk'), ('europeanCountry', 'http://www.wikidata.org/entity

In [34]:
# Now I'll retrieve the country of the place of birth
queryString = """
SELECT DISTINCT ?s ?sName ?europeanCountry ?countryName
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104 ;
       wdt:P19 ?place .
   ?europeanCountry wdt:P31 wd:Q6256 ;
               wdt:P30 wd:Q46 .
   ?place wdt:P17 ?europeanCountry .
   # get the label
   ?s sc:name ?sName .
   ?europeanCountry sc:name ?countryName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q7085'), ('sName', 'Niels Bohr'), ('europeanCountry', 'http://www.wikidata.org/entity/Q756617'), ('countryName', 'Danish Realm')]
[('s', 'http://www.wikidata.org/entity/Q103854'), ('sName', 'Aage Niels Bohr'), ('europeanCountry', 'http://www.wikidata.org/entity/Q756617'), ('countryName', 'Danish Realm')]
[('s', 'http://www.wikidata.org/entity/Q71023'), ('sName', 'Alfred Kastler'), ('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('s', 'http://www.wikidata.org/entity/Q83331'), ('sName', 'Louis de Broglie'), ('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('s', 'http://www.wikidata.org/entity/Q556543'), ('sName', 'Gérard Mourou'), ('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('s', 'http://www.wikidata.org/entity/Q187224'), ('sName', 'Pierre-Gilles de Gennes'), ('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('

In [36]:
# Now I'll group by country
queryString = """
SELECT DISTINCT ?europeanCountry ?countryName (COUNT(?s) AS ?numPhysicists)
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104 ;
       wdt:P19 ?place .
   ?europeanCountry wdt:P31 wd:Q6256 ;
               wdt:P30 wd:Q46 .
   ?place wdt:P17 ?europeanCountry .
   # get the label
   ?s sc:name ?sName .
   ?europeanCountry sc:name ?countryName .
}
GROUP BY ?europeanCountry ?countryName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('europeanCountry', 'http://www.wikidata.org/entity/Q28'), ('countryName', 'Hungary'), ('numPhysicists', '2')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('numPhysicists', '5')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q34'), ('countryName', 'Sweden'), ('numPhysicists', '4')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('numPhysicists', '10')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q184'), ('countryName', 'Belarus'), ('numPhysicists', '1')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q40'), ('countryName', 'Austria'), ('numPhysicists', '3')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q36'), ('countryName', 'Poland'), ('numPhysicists', '7')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q213'), ('countryName', 'Czech Republic'), ('numPhysicists', '1')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q29999'), ('countryName', 'Kingdom of

Final query for this task

In [37]:
# write your final query
queryString = """
SELECT DISTINCT ?europeanCountry ?countryName (COUNT(?s) AS ?numPhysicists)
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104 ;
       wdt:P19 ?place .
   ?europeanCountry wdt:P31 wd:Q6256 ;
               wdt:P30 wd:Q46 .
   ?place wdt:P17 ?europeanCountry .
   # get the label
   ?s sc:name ?sName .
   ?europeanCountry sc:name ?countryName .
}
GROUP BY ?europeanCountry ?countryName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('europeanCountry', 'http://www.wikidata.org/entity/Q28'), ('countryName', 'Hungary'), ('numPhysicists', '2')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('numPhysicists', '5')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q34'), ('countryName', 'Sweden'), ('numPhysicists', '4')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('numPhysicists', '10')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q184'), ('countryName', 'Belarus'), ('numPhysicists', '1')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q40'), ('countryName', 'Austria'), ('numPhysicists', '3')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q36'), ('countryName', 'Poland'), ('numPhysicists', '7')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q213'), ('countryName', 'Czech Republic'), ('numPhysicists', '1')]
[('europeanCountry', 'http://www.wikidata.org/entity/Q29999'), ('countryName', 'Kingdom of

## Task 5
Return the countries which won at least once all the type of Nobel Prizes with the people born there. (the result set must be a list of couples country IRI and label).

In [41]:
# write your queries
# I'll start by retrieving all the awards
queryString = """
SELECT DISTINCT ?nobelPrize ?nobelPrizeName
WHERE {
   # bind something
   ?s wdt:P166 ?nobelPrize .
   # get the label
   ?nobelPrize sc:name ?nobelPrizeName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('nobelPrize', 'http://www.wikidata.org/entity/Q4342970'), ('nobelPrizeName', 'Commemorative Decoration of the Second National Rising')]
[('nobelPrize', 'http://www.wikidata.org/entity/Q5475056'), ('nobelPrizeName', 'Anagrama Essay Prize')]
[('nobelPrize', 'http://www.wikidata.org/entity/Q6547571'), ('nobelPrizeName', 'Leelavati Award')]
[('nobelPrize', 'http://www.wikidata.org/entity/Q10891478'), ('nobelPrizeName', 'National Ten Outstanding Young Jurists')]
[('nobelPrize', 'http://www.wikidata.org/entity/Q47337762'), ('nobelPrizeName', 'Commander Grand Cross of the Order of the Lion of Finland')]
[('nobelPrize', 'http://www.wikidata.org/entity/Q5120625'), ('nobelPrizeName', 'CineMAA Awards')]
[('nobelPrize', 'http://www.wikidata.org/entity/Q5146372'), ('nobelPrizeName', 'College Football All-America Team')]
[('nobelPrize', 'http://www.wikidata.org/entity/Q53653186'), ('nobelPrizeName', 'Commander of the Order of the White Double Cross\u200e')]
[('nobelPrize', 'http://www.wiki

In [42]:
# I'll retrieve all the Nobel Prizes
queryString = """
SELECT DISTINCT ?nobelPrize ?nobelPrizeName
WHERE {
   # bind something
   ?s wdt:P166 ?nobelPrize .
   ?nobelPrize wdt:P279 wd:Q7191 .
   # get the label
   ?nobelPrize sc:name ?nobelPrizeName .
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('nobelPrize', 'http://www.wikidata.org/entity/Q35637'), ('nobelPrizeName', 'Nobel Peace Prize')]
[('nobelPrize', 'http://www.wikidata.org/entity/Q37922'), ('nobelPrizeName', 'Nobel Prize in Literature')]
[('nobelPrize', 'http://www.wikidata.org/entity/Q38104'), ('nobelPrizeName', 'Nobel Prize in Physics')]
[('nobelPrize', 'http://www.wikidata.org/entity/Q44585'), ('nobelPrizeName', 'Nobel Prize in Chemistry')]
[('nobelPrize', 'http://www.wikidata.org/entity/Q47170'), ('nobelPrizeName', 'Prize in Economic Sciences in Memory of Alfred Nobel')]
[('nobelPrize', 'http://www.wikidata.org/entity/Q80061'), ('nobelPrizeName', 'Nobel Prize in Physiology or Medicine')]
6


In [3]:
# I'll try with using FILTER EXIST for each nobel prize
queryString = """
SELECT DISTINCT ?nobelPrize ?nobelPrizeName
WHERE {
   # bind something
   ?nobelPrize wdt:P279 wd:Q7191 .
   ?s wdt:P166 ?nobelPrize ;
       wdt:P19 ?place .
   ?place wdt:P17 ?country .
   
   # get the label
   ?nobelPrize sc:name ?nobelPrizeName .
   
   FILTER EXISTS{
       ?s wdt:P166 ?nobelPrize .
       FILTER(?nobelPrize=wd:Q35637)
   }
   FILTER EXISTS{
       ?s wdt:P166 ?nobelPrize .
       FILTER(?nobelPrize=wd:Q37922)
   }
   FILTER EXISTS{
       ?s wdt:P166 ?nobelPrize .
       FILTER(?nobelPrize=wd:Q38104)
   }
   FILTER EXISTS{
       ?s wdt:P166 ?nobelPrize .
       FILTER(?nobelPrize=wd:Q44585)
   }
   FILTER EXISTS{
       ?s wdt:P166 ?nobelPrize .
       FILTER(?nobelPrize=wd:Q47170)
   }
   FILTER EXISTS{
       ?s wdt:P166 ?nobelPrize .
       FILTER(?nobelPrize=wd:Q80061)
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [5]:
# It's matching on the person, so the result is empty because there isn't any person that won each Nobel Prize.
queryString = """
SELECT DISTINCT ?nobelPrize ?nobelPrizeName
WHERE {
   # bind something
   ?nobelPrize wdt:P279 wd:Q7191 .
   #?s wdt:P166 ?nobelPrize ;
   ?s wdt:P3 wd:Q5;
       wdt:P19 ?place .
   ?place wdt:P17 ?country .
   
   # get the label
   ?nobelPrize sc:name ?nobelPrizeName .
   
   FILTER EXISTS{
       ?s wdt:P166 ?nobelPrize .
       FILTER(?nobelPrize=wd:Q35637)
   }
   FILTER EXISTS{
       ?s wdt:P166 ?nobelPrize .
       FILTER(?nobelPrize=wd:Q37922)
   }
   FILTER EXISTS{
       ?s wdt:P166 ?nobelPrize .
       FILTER(?nobelPrize=wd:Q38104)
   }
   FILTER EXISTS{
       ?s wdt:P166 ?nobelPrize .
       FILTER(?nobelPrize=wd:Q44585)
   }
   FILTER EXISTS{
       ?s wdt:P166 ?nobelPrize .
       FILTER(?nobelPrize=wd:Q47170)
   }
   FILTER EXISTS{
       ?s wdt:P166 ?nobelPrize .
       FILTER(?nobelPrize=wd:Q80061)
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [7]:
# Return the countries which won at least once all the type of Nobel Prizes with the people born there. (the result set must be a list of couples country IRI and label).
# I have to change variables names inside the FILTER EXISTS for s and place because I don't want them to match with the upper variables that are out of the filter exists.
# The only variables I want to match is country.
queryString = """
SELECT DISTINCT ?nobelPrize ?nobelPrizeName
WHERE {
   # bind something
   ?nobelPrize wdt:P279 wd:Q7191 .
   ?s wdt:P31 wd:Q5;
       wdt:P19 ?place .
   ?place wdt:P17 ?country .
   
   # get the label
   ?nobelPrize sc:name ?nobelPrizeName .
   
   FILTER EXISTS{
       ?s1 wdt:P166 ?nobelPrize;
           wdt:P19 ?place1 .
       ?place1 wdt:P17 ?country .
       FILTER(?nobelPrize=wd:Q35637)
   }
   FILTER EXISTS{
       ?s1 wdt:P166 ?nobelPrize;
           wdt:P19 ?place1 .
       ?place1 wdt:P17 ?country .
       FILTER(?nobelPrize=wd:Q37922)
   }
   FILTER EXISTS{
       ?s1 wdt:P166 ?nobelPrize;
           wdt:P19 ?place1 .
       ?place1 wdt:P17 ?country .
       FILTER(?nobelPrize=wd:Q38104)
   }
   FILTER EXISTS{
       ?s1 wdt:P166 ?nobelPrize;
           wdt:P19 ?place1 .
       ?place1 wdt:P17 ?country .
       FILTER(?nobelPrize=wd:Q44585)
   }
   FILTER EXISTS{
       ?s1 wdt:P166 ?nobelPrize;
           wdt:P19 ?place1 .
       ?place1 wdt:P17 ?country .
       FILTER(?nobelPrize=wd:Q47170)
   }
   FILTER EXISTS{
       ?s1 wdt:P166 ?nobelPrize;
           wdt:P19 ?place1 .
       ?place1 wdt:P17 ?country .
       FILTER(?nobelPrize=wd:Q80061)
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
The operation failed timed out


In [8]:
# I will try by removing the BGPs outside the filters because I understood that I don't need them.
queryString = """
SELECT DISTINCT ?country ?countryName
WHERE {
   # bind something
   ?nobelPrize wdt:P279 wd:Q7191 .
   
   # get the label
   ?nobelPrize sc:name ?nobelPrizeName .
   ?country sc:name ?countryName .
   
   FILTER EXISTS{
       ?s1 wdt:P166 ?nobelPrize;
           wdt:P19 ?place1 .
       ?place1 wdt:P17 ?country .
       FILTER(?nobelPrize=wd:Q35637)
   }
   FILTER EXISTS{
       ?s1 wdt:P166 ?nobelPrize;
           wdt:P19 ?place1 .
       ?place1 wdt:P17 ?country .
       FILTER(?nobelPrize=wd:Q37922)
   }
   FILTER EXISTS{
       ?s1 wdt:P166 ?nobelPrize;
           wdt:P19 ?place1 .
       ?place1 wdt:P17 ?country .
       FILTER(?nobelPrize=wd:Q38104)
   }
   FILTER EXISTS{
       ?s1 wdt:P166 ?nobelPrize;
           wdt:P19 ?place1 .
       ?place1 wdt:P17 ?country .
       FILTER(?nobelPrize=wd:Q44585)
   }
   FILTER EXISTS{
       ?s1 wdt:P166 ?nobelPrize;
           wdt:P19 ?place1 .
       ?place1 wdt:P17 ?country .
       FILTER(?nobelPrize=wd:Q47170)
   }
   FILTER EXISTS{
       ?s1 wdt:P166 ?nobelPrize;
           wdt:P19 ?place1 .
       ?place1 wdt:P17 ?country .
       FILTER(?nobelPrize=wd:Q80061)
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q34'), ('countryName', 'Sweden')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('countryName', 'Canada')]
[('country', 'http://www.wikidata.org/entity/Q668'), ('countryName', 'India')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy')]
[('country', 'http://www.wikidata.org/entity/Q20'), ('countryName', 'Norway')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('country', 'http://www.wikidata.org/entity/Q36'), ('countryName', 'Poland')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('countryName', 'Austria')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('countryName', 'Russia')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('countryName', 'United States of America')]
12


Final query for this task

In [9]:
# write your final query
queryString = """
SELECT DISTINCT ?country ?countryName
WHERE {
   # bind something
   ?nobelPrize wdt:P279 wd:Q7191 .
   
   # get the label
   ?nobelPrize sc:name ?nobelPrizeName .
   ?country sc:name ?countryName .
   
   FILTER EXISTS{
       ?s1 wdt:P166 ?nobelPrize;
           wdt:P19 ?place1 .
       ?place1 wdt:P17 ?country .
       FILTER(?nobelPrize=wd:Q35637)
   }
   FILTER EXISTS{
       ?s1 wdt:P166 ?nobelPrize;
           wdt:P19 ?place1 .
       ?place1 wdt:P17 ?country .
       FILTER(?nobelPrize=wd:Q37922)
   }
   FILTER EXISTS{
       ?s1 wdt:P166 ?nobelPrize;
           wdt:P19 ?place1 .
       ?place1 wdt:P17 ?country .
       FILTER(?nobelPrize=wd:Q38104)
   }
   FILTER EXISTS{
       ?s1 wdt:P166 ?nobelPrize;
           wdt:P19 ?place1 .
       ?place1 wdt:P17 ?country .
       FILTER(?nobelPrize=wd:Q44585)
   }
   FILTER EXISTS{
       ?s1 wdt:P166 ?nobelPrize;
           wdt:P19 ?place1 .
       ?place1 wdt:P17 ?country .
       FILTER(?nobelPrize=wd:Q47170)
   }
   FILTER EXISTS{
       ?s1 wdt:P166 ?nobelPrize;
           wdt:P19 ?place1 .
       ?place1 wdt:P17 ?country .
       FILTER(?nobelPrize=wd:Q80061)
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q34'), ('countryName', 'Sweden')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('countryName', 'Canada')]
[('country', 'http://www.wikidata.org/entity/Q668'), ('countryName', 'India')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy')]
[('country', 'http://www.wikidata.org/entity/Q20'), ('countryName', 'Norway')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('country', 'http://www.wikidata.org/entity/Q36'), ('countryName', 'Poland')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('countryName', 'Austria')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('countryName', 'Russia')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('countryName', 'United States of America')]
12


## Task 6
Amongst the physicists who won the Nobel Prize in Physics, find who had discovered or invented at least 3 things (the result set must be a list of triples physicist IRI, label and #discoveries ).

In [4]:
# write your queries
# I'll start by retrieving all the physicist that won the Nobel Prize in Physics Q38104 and all the discovers and inventions P61
queryString = """
SELECT DISTINCT ?s ?sName ?disc ?discName
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104 .
   ?disc wdt:P61 ?s .
   
   # get the label
   ?s sc:name ?sName .
   ?disc sc:name ?discName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q71024'), ('sName', 'Klaus von Klitzing'), ('disc', 'http://www.wikidata.org/entity/Q1047822'), ('discName', 'quantum Hall effect')]
[('s', 'http://www.wikidata.org/entity/Q71022'), ('sName', 'Ernst Ruska'), ('disc', 'http://www.wikidata.org/entity/Q132560'), ('discName', 'electron microscope')]
[('s', 'http://www.wikidata.org/entity/Q191922'), ('sName', 'Frederick Reines'), ('disc', 'http://www.wikidata.org/entity/Q1066748'), ('discName', 'antineutrino')]
[('s', 'http://www.wikidata.org/entity/Q57068'), ('sName', 'Wilhelm Wien'), ('disc', 'http://www.wikidata.org/entity/Q214336'), ('discName', "Wien's displacement law")]
[('s', 'http://www.wikidata.org/entity/Q57077'), ('sName', 'Ferdinand Braun'), ('disc', 'http://www.wikidata.org/entity/Q83913'), ('discName', 'cathode ray tube')]
[('s', 'http://www.wikidata.org/entity/Q57077'), ('sName', 'Ferdinand Braun'), ('disc', 'http://www.wikidata.org/entity/Q174320'), ('discName', 'oscilloscope')

In [6]:
#now I'll group by the inventor, so s, and count the discoveries
queryString = """
SELECT DISTINCT ?s ?sName (COUNT(?disc) as ?numDiscoveries)
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104 .
   ?disc wdt:P61 ?s .
   
   # get the label
   ?s sc:name ?sName .
   ?disc sc:name ?discName
}
GROUP BY ?s ?sName
HAVING (COUNT(?disc) >=3)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q193803'), ('sName', 'Roger Penrose'), ('numDiscoveries', '6')]
[('s', 'http://www.wikidata.org/entity/Q160518'), ('sName', 'Gustaf Dalén'), ('numDiscoveries', '3')]
[('s', 'http://www.wikidata.org/entity/Q8753'), ('sName', 'Enrico Fermi'), ('numDiscoveries', '6')]
[('s', 'http://www.wikidata.org/entity/Q123975'), ('sName', 'Michel Mayor'), ('numDiscoveries', '12')]
[('s', 'http://www.wikidata.org/entity/Q173028'), ('sName', 'Emilio G. Segrè'), ('numDiscoveries', '3')]
[('s', 'http://www.wikidata.org/entity/Q124013'), ('sName', 'Didier Queloz'), ('numDiscoveries', '3')]
[('s', 'http://www.wikidata.org/entity/Q47480'), ('sName', 'Paul Dirac'), ('numDiscoveries', '3')]
[('s', 'http://www.wikidata.org/entity/Q937'), ('sName', 'Albert Einstein'), ('numDiscoveries', '10')]
[('s', 'http://www.wikidata.org/entity/Q37463'), ('sName', 'Pierre Curie'), ('numDiscoveries', '4')]
[('s', 'http://www.wikidata.org/entity/Q47285'), ('sName', 'J. J. Thomson

Final query for this task

In [7]:
# write your final query
#now I'll group by the inventor, so s, and count the discoveries
queryString = """
SELECT DISTINCT ?s ?sName (COUNT(?disc) as ?numDiscoveries)
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104 .
   ?disc wdt:P61 ?s .
   
   # get the label
   ?s sc:name ?sName .
   ?disc sc:name ?discName
}
GROUP BY ?s ?sName
HAVING (COUNT(?disc) >=3)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q193803'), ('sName', 'Roger Penrose'), ('numDiscoveries', '6')]
[('s', 'http://www.wikidata.org/entity/Q160518'), ('sName', 'Gustaf Dalén'), ('numDiscoveries', '3')]
[('s', 'http://www.wikidata.org/entity/Q8753'), ('sName', 'Enrico Fermi'), ('numDiscoveries', '6')]
[('s', 'http://www.wikidata.org/entity/Q123975'), ('sName', 'Michel Mayor'), ('numDiscoveries', '12')]
[('s', 'http://www.wikidata.org/entity/Q173028'), ('sName', 'Emilio G. Segrè'), ('numDiscoveries', '3')]
[('s', 'http://www.wikidata.org/entity/Q124013'), ('sName', 'Didier Queloz'), ('numDiscoveries', '3')]
[('s', 'http://www.wikidata.org/entity/Q47480'), ('sName', 'Paul Dirac'), ('numDiscoveries', '3')]
[('s', 'http://www.wikidata.org/entity/Q937'), ('sName', 'Albert Einstein'), ('numDiscoveries', '10')]
[('s', 'http://www.wikidata.org/entity/Q37463'), ('sName', 'Pierre Curie'), ('numDiscoveries', '4')]
[('s', 'http://www.wikidata.org/entity/Q47285'), ('sName', 'J. J. Thomson

## Task 7
Amongst the physicists who won the Nobel Prize in Physics, find who taught in the university of his/her city of birth (the result set must be a list of elements of the form physicist IRI, label, university IRI and label).

In [9]:
# write your queries
# Among the winners of nobel prize in Physics, I'll start by retrieving the ones that has an occupation as university teacher Q1622272
queryString = """
SELECT DISTINCT ?s ?sName ?university ?universityName
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104 ;
       wdt:P106 wd:Q1622272 ;
       wdt:P108 ?university ;
       wdt:P19 ?placeOfBirth .
   
   # get the label
   ?s sc:name ?sName .
   ?university sc:name ?universityName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q133747'), ('sName', 'William Henry Bragg'), ('university', 'http://www.wikidata.org/entity/Q15574'), ('universityName', 'University of Adelaide')]
[('s', 'http://www.wikidata.org/entity/Q181363'), ('sName', 'Brian David Josephson'), ('university', 'http://www.wikidata.org/entity/Q4380853'), ('universityName', 'Missouri University of Science and Technology')]
[('s', 'http://www.wikidata.org/entity/Q41269'), ('sName', 'Henri Becquerel'), ('university', 'http://www.wikidata.org/entity/Q838691'), ('universityName', "Muséum national d'histoire naturelle")]
[('s', 'http://www.wikidata.org/entity/Q71013'), ('sName', 'Theodor W. Hänsch'), ('university', 'http://www.wikidata.org/entity/Q829535'), ('universityName', 'Max Planck Institute of Quantum Optics')]
[('s', 'http://www.wikidata.org/entity/Q57100'), ('sName', 'Maria Goeppert Mayer'), ('university', 'http://www.wikidata.org/entity/Q649120'), ('universityName', 'Argonne National Laboratory')]


In [2]:
#I'll try to retrieve the place of the University of Adelaide Q15574
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q15574 ?p ?obj .
   
   # get the label
   ?p sc:name ?pName .
   ?obj sc:name ?objName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P488'), ('pName', 'chairperson'), ('obj', 'http://www.wikidata.org/entity/Q1740252'), ('objName', 'Kevin Scarce')]
[('p', 'http://www.wikidata.org/prop/direct/P2936'), ('pName', 'language used'), ('obj', 'http://www.wikidata.org/entity/Q1860'), ('objName', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('pName', 'member of'), ('obj', 'http://www.wikidata.org/entity/Q19861084'), ('objName', 'ORCID, Inc.')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('obj', 'http://www.wikidata.org/entity/Q334'), ('objName', 'Singapore')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q408'), ('objName', 'Australia')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('pName', 'member of'), ('obj', 'http://www.wikidata.org/entity/Q5275956'), ('objName', 'Digital Preservation Coalition')]
[('p', 'http://www.wikidata.org/prop/direct/P1343')

In [11]:
# I have to take the locations that are cities.
# instance of P31 city Q515
queryString = """
SELECT DISTINCT ?s ?sName ?university ?universityName ?city ?cityName
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104 ;
       wdt:P106 wd:Q1622272 ;
       wdt:P108 ?university ;
       wdt:P19 ?city .
   ?city wdt:P31 wd:Q515 .
   ?university ?p ?city .
   
   # get the label
   ?s sc:name ?sName .
   ?university sc:name ?universityName .
   ?city sc:name ?cityName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q44286'), ('sName', 'Rudolf Mössbauer'), ('university', 'http://www.wikidata.org/entity/Q157808'), ('universityName', 'Technical University of Munich'), ('city', 'http://www.wikidata.org/entity/Q1726'), ('cityName', 'Munich')]
[('s', 'http://www.wikidata.org/entity/Q184499'), ('sName', 'Patrick Blackett, Baron Blackett'), ('university', 'http://www.wikidata.org/entity/Q375606'), ('universityName', 'Birkbeck, University of London'), ('city', 'http://www.wikidata.org/entity/Q84'), ('cityName', 'London')]
2


In [12]:
# I'll understand better what is the predicate that links the university to the city
queryString = """
SELECT DISTINCT ?s ?sName ?university ?universityName ?p ?pName ?city ?cityName
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104 ;
       wdt:P106 wd:Q1622272 ;
       wdt:P108 ?university ;
       wdt:P19 ?city .
   ?city wdt:P31 wd:Q515 .
   ?university ?p ?city .
   
   # get the label
   ?s sc:name ?sName .
   ?p sc:name ?pName .
   ?university sc:name ?universityName .
   ?city sc:name ?cityName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q44286'), ('sName', 'Rudolf Mössbauer'), ('university', 'http://www.wikidata.org/entity/Q157808'), ('universityName', 'Technical University of Munich'), ('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('city', 'http://www.wikidata.org/entity/Q1726'), ('cityName', 'Munich')]
[('s', 'http://www.wikidata.org/entity/Q44286'), ('sName', 'Rudolf Mössbauer'), ('university', 'http://www.wikidata.org/entity/Q157808'), ('universityName', 'Technical University of Munich'), ('p', 'http://www.wikidata.org/prop/direct/P159'), ('pName', 'headquarters location'), ('city', 'http://www.wikidata.org/entity/Q1726'), ('cityName', 'Munich')]
[('s', 'http://www.wikidata.org/entity/Q184499'), ('sName', 'Patrick Blackett, Baron Blackett'), ('university', 'http://www.wikidata.org/entity/Q375606'), ('universityName', 'Birkbeck, University of London'), ('p', 'http://www.wikidata.org/prop/direct/P159'),

In [13]:
# I'll try to see if there are headquarters that are specific zone of a city
queryString = """
SELECT DISTINCT ?s ?sName ?university ?universityName ?p ?pName ?city ?cityName
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104 ;
       wdt:P106 wd:Q1622272 ;
       wdt:P108 ?university ;
       wdt:P19 ?city .
   ?city wdt:P31 wd:Q515 .
   {
       ?university ?p ?city .
   }
   UNION
   {
       ?university wdt:P159 ?headquarter .
       ?headquarter ?p1 ?city .
   }
   # get the label
   ?s sc:name ?sName .
   ?p sc:name ?pName .
   ?university sc:name ?universityName .
   ?city sc:name ?cityName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q44286'), ('sName', 'Rudolf Mössbauer'), ('university', 'http://www.wikidata.org/entity/Q157808'), ('universityName', 'Technical University of Munich'), ('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('city', 'http://www.wikidata.org/entity/Q1726'), ('cityName', 'Munich')]
[('s', 'http://www.wikidata.org/entity/Q44286'), ('sName', 'Rudolf Mössbauer'), ('university', 'http://www.wikidata.org/entity/Q157808'), ('universityName', 'Technical University of Munich'), ('p', 'http://www.wikidata.org/prop/direct/P159'), ('pName', 'headquarters location'), ('city', 'http://www.wikidata.org/entity/Q1726'), ('cityName', 'Munich')]
[('s', 'http://www.wikidata.org/entity/Q184499'), ('sName', 'Patrick Blackett, Baron Blackett'), ('university', 'http://www.wikidata.org/entity/Q375606'), ('universityName', 'Birkbeck, University of London'), ('p', 'http://www.wikidata.org/prop/direct/P159'),

In [15]:
# I'll remove the select of the predicate
queryString = """
SELECT DISTINCT ?s ?sName ?university ?universityName ?city ?cityName
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104 ;
       wdt:P106 wd:Q1622272 ;
       wdt:P108 ?university ;
       wdt:P19 ?city .
   ?city wdt:P31 wd:Q515 .
   {
       ?university ?p ?city .
   }
   UNION
   {
       ?university wdt:P159 ?headquarter .
       ?headquarter ?p1 ?city .
   }
   # get the label
   ?s sc:name ?sName .
   ?p sc:name ?pName .
   ?university sc:name ?universityName .
   ?city sc:name ?cityName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q44286'), ('sName', 'Rudolf Mössbauer'), ('university', 'http://www.wikidata.org/entity/Q157808'), ('universityName', 'Technical University of Munich'), ('city', 'http://www.wikidata.org/entity/Q1726'), ('cityName', 'Munich')]
[('s', 'http://www.wikidata.org/entity/Q184499'), ('sName', 'Patrick Blackett, Baron Blackett'), ('university', 'http://www.wikidata.org/entity/Q375606'), ('universityName', 'Birkbeck, University of London'), ('city', 'http://www.wikidata.org/entity/Q84'), ('cityName', 'London')]
[('s', 'http://www.wikidata.org/entity/Q189741'), ('sName', 'Melvin Schwartz'), ('university', 'http://www.wikidata.org/entity/Q49088'), ('universityName', 'Columbia University'), ('city', 'http://www.wikidata.org/entity/Q60'), ('cityName', 'New York City')]
[('s', 'http://www.wikidata.org/entity/Q192695'), ('sName', 'Leon M. Lederman'), ('university', 'http://www.wikidata.org/entity/Q49088'), ('universityName', 'Columbia University'), ('cit

In [16]:
# I'll be more general on the predicate that link university to its city
queryString = """
SELECT DISTINCT ?s ?sName ?university ?universityName ?city ?cityName
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104 ;
       wdt:P106 wd:Q1622272 ;
       wdt:P108 ?university ;
       wdt:P19 ?city .
   ?city wdt:P31 wd:Q515 .
   {
       ?university ?p ?city .
   }
   UNION
   {
       ?university ?p2 ?headquarter .
       ?headquarter ?p1 ?city .
   }
   # get the label
   ?s sc:name ?sName .
   ?p sc:name ?pName .
   ?university sc:name ?universityName .
   ?city sc:name ?cityName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
The operation failed timed out


Final query for this task

In [17]:
# write your final query
# I'll use the penultimate query 
queryString = """
SELECT DISTINCT ?s ?sName ?university ?universityName
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104 ;
       wdt:P106 wd:Q1622272 ;
       wdt:P108 ?university ;
       wdt:P19 ?city .
   ?city wdt:P31 wd:Q515 .
   {
       ?university ?p ?city .
   }
   UNION
   {
       ?university wdt:P159 ?headquarter .
       ?headquarter ?p1 ?city .
   }
   # get the label
   ?s sc:name ?sName .
   ?p sc:name ?pName .
   ?university sc:name ?universityName .
   ?city sc:name ?cityName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q44286'), ('sName', 'Rudolf Mössbauer'), ('university', 'http://www.wikidata.org/entity/Q157808'), ('universityName', 'Technical University of Munich')]
[('s', 'http://www.wikidata.org/entity/Q184499'), ('sName', 'Patrick Blackett, Baron Blackett'), ('university', 'http://www.wikidata.org/entity/Q375606'), ('universityName', 'Birkbeck, University of London')]
[('s', 'http://www.wikidata.org/entity/Q189741'), ('sName', 'Melvin Schwartz'), ('university', 'http://www.wikidata.org/entity/Q49088'), ('universityName', 'Columbia University')]
[('s', 'http://www.wikidata.org/entity/Q192695'), ('sName', 'Leon M. Lederman'), ('university', 'http://www.wikidata.org/entity/Q49088'), ('universityName', 'Columbia University')]
[('s', 'http://www.wikidata.org/entity/Q186481'), ('sName', 'Riccardo Giacconi'), ('university', 'http://www.wikidata.org/entity/Q193727'), ('universityName', 'Johns Hopkins University')]
[('s', 'http://www.wikidata.org/entity/Q18